# Fetch Data

__The Purpose of this notebook is to fetch data from the halite game and obtain qualitative intuitions; we want to visualize gameplay data__

## Load Dependencies

In [1]:
from kaggle_environments import make

## Determine Datasources

Where can we get the data from? Since this is a game; we can obtain information from (1) the game definition and (2) gameplay.

### Use Kaggle Users Dataset

A user David1013 wrote a scraper and has iteratively pulled data into a Kaggle Dataset here: https://www.kaggle.com/david1013/halite-top-games

You can find his scraper here: https://www.kaggle.com/david1013/halite-game-scraper/

### Game Definition

In [2]:
env = make("halite", debug=True)

In [3]:
env.specification.action

{'description': 'Actions taken per asset (ship or shipyard).',
 'type': 'object',
 'additionalProperties': {'description': 'Key: uid of the asset. Value: action. SPAWN for shipyards, remaining for ships.',
  'enum': ['CONVERT', 'SPAWN', 'NORTH', 'SOUTH', 'EAST', 'WEST']},
 'default': 0}

In [4]:
env.specification.observation

{'halite': {'description': 'Serialized list of available halite per cell on the board.',
  'type': 'array',
  'shared': True},
 'players': {'description': 'List of players and their assets.',
  'type': 'array',
  'shared': True},
 'player': {'description': "The current agent's player index.",
  'type': 'integer',
  'defaults': [0, 1, 2, 3]},
 'step': {'description': 'The current step index within the episode.',
  'type': 'integer',
  'shared': True,
  'minimum': 0}}

In [5]:
env.specification.configuration

{'episodeSteps': {'description': 'Maximum number of steps in the episode.',
  'type': 'integer',
  'minimum': 1,
  'default': 400},
 'agentExec': {'description': 'How the agent is executed alongside the running envionment.',
  'type': 'string',
  'default': 'LOCAL',
  'enum': ['LOCAL', 'PROCESS']},
 'agentTimeout': {'description': 'Maximum runtime (seconds) to initialize an agent.',
  'type': 'integer',
  'minimum': 1,
  'default': 30},
 'actTimeout': {'description': 'Maximum runtime (seconds) to obtain an action from an agent.',
  'type': 'integer',
  'minimum': 1,
  'default': 6},
 'runTimeout': {'description': 'Maximum runtime (seconds) of an episode (not necessarily DONE).',
  'type': 'integer',
  'minimum': 1,
  'default': 9600},
 'startingHalite': {'description': 'The starting amount of halite available on the board.',
  'type': 'integer',
  'default': 24000},
 'size': {'description': 'The number of cells vertically and horizontally on the board.',
  'type': 'integer',
  'default

### Scrape JSON from Kaggle

In [1]:
base_url = 'https://www.kaggle.com/c/halite/leaderboard'

In [6]:
from kaggle_environments import get_episode_replay, list_episodes, list_episodes_for_team, list_episodes_for_submission

In [7]:
# Returns metadata for all episodes passed by id
list_episodes([100000, 100001, 100002])

{'result': {'episodes': [{'id': 100000,
    'competitionId': 17592,
    'createTime': {'seconds': 1580104752, 'nanos': 208496000},
    'endTime': {'seconds': 1580104776, 'nanos': 202911600},
    'replayUrl': 'gs://kaggle-episode-replays/100000.json',
    'adminNotes': None,
    'state': 'completed',
    'type': 'public',
    'agents': [{'id': 197423,
      'submissionId': 14115616,
      'reward': 0.0,
      'index': 0,
      'initialScore': 703.2480497822609,
      'initialConfidence': 20.0,
      'updatedScore': 701.9174567483943,
      'updatedConfidence': 20.0,
      'state': 'unspecified',
      'submission': {'id': 14115616,
       'title': '',
       'details': '',
       'submittedUserId': 0,
       'dateSubmitted': {'seconds': 1578693663, 'nanos': 280000000},
       'teamId': 4212968,
       'privateScore': None,
       'publicScore': 0.0,
       'isSelected': False,
       'status': 'pending',
       'isAfterDeadline': False,
       'publicScoreAdditionalDetails': '',
       

In [10]:
# Returns metadata for all episodes for a particular submission
# You can find your submission id at the top of the list of episodes for your agent on the leaderboard
list_episodes_for_submission(100000)

{'result': {'episodes': [], 'teams': []}, 'wasSuccessful': True}

In [12]:
# Returns metadata for all episodes for a particular team
# You can get your team id by calling the list_episodes_for_submission method with one of your submissions
list_episodes_for_team(100003)

{'result': {'episodes': [], 'teams': []}, 'wasSuccessful': True}

In [13]:
# Returns the episode replay json for a particular episode id
get_episode_replay(100000)

{'result': {'replay': '{"configuration": {"columns": 7, "inarow": 4, "rows": 6, "steps": 1000, "timeout": 5}, "description": "Classic Connect in a row but configurable.", "name": "connectx", "rewards": [0, 1], "schema_version": 1, "specification": {"action": {"default": 0, "description": "Column to drop a checker onto the board.", "minimum": 0, "type": "integer"}, "agents": [2], "configuration": {"columns": {"default": 7, "description": "The number of columns on the board", "minimum": 1, "type": "integer"}, "inarow": {"default": 4, "description": "The number of checkers in a row required to win.", "minimum": 1, "type": "integer"}, "rows": {"default": 6, "description": "The number of rows on the board", "minimum": 1, "type": "integer"}, "steps": {"default": 1000, "description": "Maximum number of steps the environment can run.", "minimum": 1, "type": "integer"}, "timeout": {"default": 5, "description": "Seconds an agent can run before timing out.", "minimum": 1, "type": "integer"}}, "in

### Generate Gameplay Data

In [75]:
env = make("halite", debug=True)
trainer = env.train([None, "random", "random", "random"])
obs = trainer.reset()

In [76]:
hist = {
    'obs': list(),
    'reward': list(),
    'done': list(),
    'info': list(),
}

In [77]:
done = False
while not done:
    action = 'NORTH' # Action for the agent being trained.
    obs, reward, done, info = trainer.step(action)
    hist['obs'].append(obs)
    hist['reward'].append(reward)
    hist['done'].append(done)
    hist['info'].append(info)

In [78]:
env.render()

In [84]:
hist['obs'][0].keys()

dict_keys(['halite', 'players', 'player', 'step'])

In [98]:
hist['obs'][10]['players']

[[5000, {}, {'0-1': [110, 0]}],
 [4000, {'4-1': 101}, {'5-1': [58, 18]}],
 [3535, {'1-1': 320}, {'2-1': [321, 0], '8-1': [297, 0]}],
 [3000, {'4-2': 328}, {'5-2': [349, 26], '7-1': [284, 0], '11-1': [328, 0]}]]

In [ ]:
env = make("halite", debug=True)
env.run(["random", "random", "random", "random"])
env.render(mode="ipython", width=800, height=600)